## IMPORTS

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive
drive.mount('/content/gdrive')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Import necessary libraries

import torch

# Load pre-trained BERT model and tokenizer
trainFolder = "gdrive/My Drive/MLDM - Carrefour Project/data-train"

import os
for dirname, _, filenames in os.walk(f'{trainFolder}'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/gdrive
gdrive/My Drive/MLDM - Carrefour Project/data-train/products_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/test_data.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_1.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_10.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_2.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_3.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_4.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_5.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_6.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_7.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_8.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/train_data_part_9.csv
gdrive/My Drive/MLDM - Carrefour Project/data-train/products_embe.csv
gdrive/My Drive/MLDM - Car

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### Data Description

This project uses three datasets:

#### `train_data.csv`:
This dataset contains two years (2022 & 2023) of historical transactions for 100,000 Carrefour customers. It has 10 columns:

* ***date***: Date of the transaction
* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: Product purchased
* ***has_loyality_card***: Flag indicating whether the customer has a loyalty card
* ***store_id***: Store where the purchase was made
* ***is_promo***: Flag indicating whether there was a discount on the product
* ***quantity***: Quantity purchased of the product
* ***format***: Ecommerce activity format (clcv, lex, or DRIVE)
  - clcv : courses livrées chez vous
  - lex : livraison express
  - DRIVE.
* ***orderChannelCode***: Indicates whether the online activity was made through the website or mobile app

#### `products_data.csv`:
This dataset contains detailed information about the products. The following columns are relevant to this project:

* ***product_id*** : Product name
* ***product_description*** : Product description
* ***department_key***: Department key
* ***class_key***: Class key
* ***subclass_key***: Subclass key
* ***sector***: sector name
* ***brand_key***: Brand name
* ***shelf_level1***: Top-level shelf category
* ***shelf_level2***: Second-level shelf category
* ***shelf_level3***: Third-level shelf category
* ***shelf_level4***: Fourth-level shelf category
* ***sector***: Sector
* ***bio***: Flag indicating whether the product is organic
* ***sugar_free***: Flag indicating whether the product is sugar-free
* ***aspartame_free***: Flag indicating whether the product is aspartame-free
* ***gluten_free***: Flag indicating whether the product is gluten-free
* ***halal***: Flag indicating whether the product is halal
* ***casher***: Flag indicating whether the product is kosher
* ***eco_friendly***: Flag indicating whether the product is eco-friendly
* ***local_french***: Flag indicating whether the product is locally produced in France
* ***artificial_coloring_free***: Flag indicating whether the product is free of artificial coloring
* ***taste_enhancer_free***: Flag indicating whether the product is free of taste enhancers
* ***naturality***: Naturality score
* ***antibiotic_free***: Flag indicating whether the product is antibiotic-free
* ***reduced_sugar***: Flag indicating whether the product has reduced sugar content
* ***vegetarian***: Flag indicating whether the product is vegetarian
* ***pesticide_free***: Flag indicating whether the product is pesticide-free
* ***grain_free***: Flag indicating whether the product is grain-free
* ***no_added_sugar***: Flag indicating whether the product has no added sugar
* ***salt_reduced***: Flag indicating whether the product has reduced salt content
* ***nitrite_free***: Flag indicating whether the product is nitrite-free
* ***fed_without_ogm***: Flag indicating whether the animals were fed without GMOs
* ***no_added_salt***: Flag indicating whether the product has no added salt
* ***no_artificial_flavours***: Flag indicating whether the product has no artificial flavors
* ***porc***: Flag indicating whether the product contains pork
* ***vegan***: Flag indicating whether the product is vegan
* ***frozen***: Flag indicating whether the product is frozen
* ***fat_free***: Flag indicating whether the product is fat-free
* ***reduced_fats***: Flag indicating whether the product has reduced fat content
* ***fresh***: Flag indicating whether the product is fresh
* ***alcool***: Flag indicating whether the product contains alcohol
* ***lactose_free***: Flag indicating whether the product is lactose-free
* ***phenylalanine_free***: Flag indicating whether the product is phenylalanine-free
* ***palm_oil_free***: Flag indicating whether the product is palm oil-free
* ***ecoscore***: Ecoscore
* ***produits_du_monde***: Flag indicating whether the product is an international product
* ***regional_product***: Flag indicating whether the product is a regional product
* ***national_brand***: Flag indicating whether the product is a national brand
* ***first_price_brand***: Flag indicating whether the product is a first-price brand
* ***carrefour_brand***: Flag indicating whether the product is a Carrefour brand

#### `test_data.csv`:
This dataset contains the actual purchases of the first 80,000 customers in 2024. It has three columns:

* ***transaction_id***: ID of the transaction
* ***customer_id***: Customer ID
* ***product_id***: the id of the purchased product

### Load data

* Load *train_data.csv*, *products_data.csv* and *test_data.csv* using pandas.

In [3]:
# This code reads the data from CSV files named "train_data_part_i.csv" for all i from 1 to 10
# and concatenates them into a single pandas DataFrame
train_dataframes = []
for i in tqdm(range(1, 11)): ## 1 - 11
    train_dataframes.append(pd.read_csv(f'{trainFolder}/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

100%|██████████| 10/10 [02:04<00:00, 12.42s/it]


In [4]:
#train_data = train_data.sample(frac=0.1, random_state=1)  # frac=0.1 takes 10% of the data

In [5]:
# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv(f'{trainFolder}/products_data.csv')

<ipython-input-5-d8cc84ca7717>:2: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv(f'{trainFolder}/products_data.csv')


In [6]:
# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv(f'{trainFolder}/test_data.csv')

In [7]:
# Load product embeddings
product_embeddings_np = np.load(f'{trainFolder}/product_embeddings.npy')

# Load product IDs
product_ids = np.load(f'{trainFolder}/product_ids.npy', allow_pickle=True)  # Use allow_pickle=True if product_ids is an array of objects (e.g., strings)

# Create the product embedding dictionary
product_embedding_dict = dict(zip(product_ids, product_embeddings_np))

In [8]:
train_data

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
0,2023-11-15,Transaction_1730125,Household_39,Product_5362,0,Store_2,0,1.0,DRIVE,MOBILE_APP
1,2022-07-20,Transaction_1560535,Household_39,Product_67174,0,Store_2,0,2.0,DRIVE,WEBSITE
2,2022-07-20,Transaction_1560535,Household_39,Product_82254,0,Store_2,0,2.0,DRIVE,WEBSITE
3,2023-11-15,Transaction_1730125,Household_39,Product_3895,0,Store_2,0,1.0,DRIVE,MOBILE_APP
4,2022-07-20,Transaction_1560535,Household_39,Product_34014,0,Store_2,0,1.0,DRIVE,WEBSITE
...,...,...,...,...,...,...,...,...,...,...
87037457,2022-01-07,Transaction_1162379,Household_96742,Product_28756,0,Store_2542,0,1.0,DRIVE,MOBILE_APP
87037458,2023-08-17,Transaction_2306043,Household_96742,Product_62786,0,Store_2542,0,1.0,DRIVE,MOBILE_APP
87037459,2022-10-12,Transaction_1524531,Household_96742,Product_16362,0,Store_2542,0,1.0,DRIVE,MOBILE_APP
87037460,2023-03-10,Transaction_1972306,Household_96742,Product_78870,0,Store_2542,0,1.0,DRIVE,MOBILE_APP


In [9]:
# prompt: print from train_date rows with customer id Household_80001
import pandas as pd

# Create a list of customer IDs within the specified range
#customer_ids_new = [f'Household_{i}' for i in range(80001, 100002)]
#customer_ids_new = [f'Household_{i}' for i in range(10001, 20000)]
customer_ids_new = [f'Household_{i}' for i in range(10001, 20000)]

# prompt: keep in train_data and test_data the rows that have the cusomer_id in customer_ids

# Filter train_data
train_data = train_data[train_data['customer_id'].isin(customer_ids_new)]

# Filter test_data
test_data = test_data[test_data['customer_id'].isin(customer_ids_new)]

In [10]:
train_data

,date,transaction_id,customer_id,product_id,has_loyality_card,store_id,is_promo,quantity,format,order_channel
8757405,2023-01-14,Transaction_362590,Household_10188,Product_62613,0,Store_2,0,1.0,DRIVE,WEBSITE
8757406,2022-05-29,Transaction_2466178,Household_10188,Product_71119,0,Store_2,0,1.0,DRIVE,WEBSITE
8757407,2022-05-29,Transaction_2466178,Household_10188,Product_23971,0,Store_2,0,2.0,DRIVE,WEBSITE
8757408,2022-04-16,Transaction_1898817,Household_10188,Product_3907,0,Store_2,0,1.0,DRIVE,WEBSITE
8757409,2023-12-10,Transaction_147391,Household_10188,Product_37906,0,Store_2,0,1.0,DRIVE,WEBSITE
...,...,...,...,...,...,...,...,...,...,...
17514309,2022-08-19,Transaction_1545017,Household_19505,Product_62926,0,Store_2542,0,1.0,DRIVE,MOBILE_APP
17514310,2023-10-17,Transaction_318974,Household_11112,Product_75788,0,Store_2544,0,1.0,LEX,LIVREE
17514311,2023-11-25,Transaction_906692,Household_11112,Product_46620,0,Store_2544,0,1.0,LEX,LIVREE
17514312,2023-07-07,Transaction_1176748,Household_11112,Product_27326,0,Store_2544,0,1.0,LEX,LIVREE


In [11]:
test_data

,transaction_id,customer_id,product_id
0,Transaction_2024_1,Household_16874,Product_9790
1,Transaction_2024_1,Household_16874,Product_68295
2,Transaction_2024_1,Household_16874,Product_19494
3,Transaction_2024_1,Household_16874,Product_11109
22,Transaction_2024_9,Household_17896,Product_74083
...,...,...,...
915448,Transaction_2024_55287,Household_19999,Product_52779
915449,Transaction_2024_55287,Household_19999,Product_34511
915450,Transaction_2024_55287,Household_19999,Product_53550
915451,Transaction_2024_55287,Household_19999,Product_1370


In [12]:
train_data = train_data.drop(columns=['transaction_id','has_loyality_card', 'store_id','format','order_channel'])

## Training Data Preproccesing

In [13]:
train_data['date'] = pd.to_datetime(train_data['date'])

# Add recency attribute
latest_date = train_data['date'].max()  # Find the latest date in the dataset
train_data['recency'] = (latest_date - train_data['date']).dt.days  # Calculate days since last purchase

# Group by customer_id and product_id to calculate quantity and most recent purchase
customer_product_data = train_data.groupby(['customer_id', 'product_id']).agg({
    'quantity': 'sum',
    'recency': 'min'  # Minimum days since purchase (most recent)
}).reset_index()

# Normalize quantity and recency scores
customer_product_data['quantity_score'] = customer_product_data['quantity'] / customer_product_data['quantity'].max()
customer_product_data['recency_score'] = 1 - (customer_product_data['recency'] / customer_product_data['recency'].max())  # Recent = higher score

best_alpha = 0.97

# Calculate the final score with this alpha value
customer_product_data['final_score'] = (
    best_alpha * customer_product_data['quantity_score'] +
    (1 - best_alpha) * customer_product_data['recency_score']
)

# Rank products for each customer with unique ranks
customer_product_data['rank'] = customer_product_data.sort_values(
    by=['customer_id', 'final_score', 'quantity', 'product_id'],  # Add secondary sort keys
    ascending=[True, False, False, True]  # Ensure consistent order
).groupby('customer_id').cumcount() + 1  # Assign unique incremental ranks


# Sort the DataFrame by customer_id and rank
customer_product_data = customer_product_data.sort_values(by=['customer_id', 'rank'])

# Filter out products with a rank greater than 200 for each customer
#customer_product_data = customer_product_data[customer_product_data['rank'] <= 20]


In [14]:
customer_product_data

,customer_id,product_id,quantity,recency,quantity_score,recency_score,final_score,rank
107,Household_10001,Product_57942,40.0,15,0.015649,0.979424,0.044563,1
127,Household_10001,Product_65600,39.0,8,0.015258,0.989026,0.044471,2
60,Household_10001,Product_35862,38.0,8,0.014867,0.989026,0.044092,3
51,Household_10001,Product_32175,29.0,15,0.011346,0.979424,0.040388,4
30,Household_10001,Product_21737,28.0,8,0.010955,0.989026,0.040297,5
...,...,...,...,...,...,...,...,...
3363809,Household_19999,Product_25509,1.0,620,0.000391,0.149520,0.004865,616
3364059,Household_19999,Product_53731,1.0,620,0.000391,0.149520,0.004865,617
3364120,Household_19999,Product_60476,1.0,620,0.000391,0.149520,0.004865,618
3364233,Household_19999,Product_7641,1.0,620,0.000391,0.149520,0.004865,619


In [15]:


# Extract temporal features
train_data['month'] = train_data['date'].dt.month
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['is_weekend'] = train_data['day_of_week'].isin([5, 6]).astype(int)

# Calculate recency: days since the last purchase for each customer
train_data['days_since_last_purchase'] = train_data.groupby('customer_id')['date'].transform(
    lambda x: (x.max() - x).dt.days
)

# Drop the 'date' column if it's no longer needed
train_data = train_data.drop(columns=['date'], errors='ignore')

In [16]:
# Aggregate customer-specific features
customer_features = train_data.groupby(['customer_id']).agg(
    purchase_frequency=('product_id', 'count'),  # Total number of purchases
    avg_quantity=('quantity', 'mean'),          # Average quantity purchased
    promo_ratio=('is_promo', 'mean'),           # Proportion of purchases made with promotions
    unique_products=('product_id', 'nunique')   # Number of unique products purchased
).reset_index()

# Merge these features into train_data and test_data
train_data = train_data.merge(customer_features, on='customer_id', how='left')

In [17]:
train_data.tail()

,customer_id,product_id,is_promo,quantity,recency,month,day_of_week,is_weekend,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
8756541,Household_19505,Product_62926,0,1.0,499,8,4,0,490,381,1.094488,0.028871,270
8756542,Household_11112,Product_75788,0,1.0,75,10,1,0,50,370,1.186486,0.029730,227
8756543,Household_11112,Product_46620,0,1.0,36,11,5,1,11,370,1.186486,0.029730,227
8756544,Household_11112,Product_27326,0,1.0,177,7,4,0,152,370,1.186486,0.029730,227
8756545,Household_11112,Product_5415,0,1.0,75,10,1,0,50,370,1.186486,0.029730,227


In [18]:
products_data.tail()

,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
82961,Product_18949,857699 BONNET POLAIRE NOIR TU,Department_66,Class_6620,SubClass_66204,TEXTILE,TEX,Mode et Bagagerie,Bagagerie et Maroquinerie,Accessoires de mode,"Chapeaux, Casquettes et Bonnets",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82962,Product_66524,857699 BONNET PO BLANC CAS2 TU,Department_66,Class_6620,SubClass_66204,TEXTILE,TEX,Mode et Bagagerie,Bagagerie et Maroquinerie,Accessoires de mode,"Chapeaux, Casquettes et Bonnets",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82963,Product_66126,876946 PULL DE NOEL PET ECRU S,Department_66,Class_6635,SubClass_66358,TEXTILE,TEX,Animalerie,Chiens,"Colliers, Laisses et Vêtements",Vêtements et Chaussures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82964,Product_31161,CHARGE BUT 13K CRF,Department_75,Class_7520,SubClass_75201,ACTIVITES PERIPHERIQUES,CARREFOUR,Maison et Décoration,Bouteilles de gaz,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82965,Product_80183,CHARGE BUT 6K AZ,Department_75,Class_7520,SubClass_75203,ACTIVITES PERIPHERIQUES,ZZZZZZZZZZ,Maison et Décoration,Bouteilles de gaz,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1


# Identifyng Missing Products

In [19]:
# Convert product_id columns to sets for quick set operations
train_product_ids = set(train_data["product_id"].unique())
products_product_ids = set(products_data["product_id"].unique())

# Find products in products_data that are not in train_data
missing_from_train = products_product_ids - train_product_ids

print(f"Number of products not in train_data: {len(missing_from_train)}")

if len(missing_from_train) > 0:
    print("These product IDs are in products_data but not in train_data:")
    for pid in missing_from_train:
        print(pid)
else:
    print("All products in products_data appear in train_data.")


Se han truncado las últimas 5000 líneas del flujo de salida.
Product_41663
Product_67055
Product_6037
Product_21420
Product_40138
Product_62962
Product_57460
Product_65025
Product_69166
Product_23440
Product_65509
Product_5729
Product_76098
Product_12992
Product_49999
Product_26162
Product_81415
Product_80068
Product_59680
Product_2955
Product_17382
Product_39052
Product_24858
Product_47331
Product_27423
Product_42335
Product_3691
Product_81377
Product_76831
Product_18475
Product_7108
Product_12216
Product_78118
Product_72779
Product_43318
Product_80261
Product_42524
Product_34942
Product_3994
Product_20280
Product_56234
Product_11354
Product_52021
Product_17112
Product_26041
Product_7989
Product_6056
Product_71795
Product_66389
Product_13000
Product_36739
Product_18716
Product_710
Product_19321
Product_78113
Product_29541
Product_30087
Product_19389
Product_28771
Product_70351
Product_26077
Product_17233
Product_25846
Product_52888
Product_78157
Product_25688
Product_32667
Product_526

In [20]:
missing_products_info = products_data[products_data["product_id"].isin(missing_from_train)]

# Display the missing products dataframe
missing_products_info


,product_id,product_description,department_key,class_key,subclass_key,sector,brand_key,shelf_level1,shelf_level2,shelf_level3,shelf_level4,bio,sugar_free,aspartame_free,gluten_free,halal,casher,eco_friendly,local_french,artificial_coloring_free,taste_enhancer_free,naturality,antibiotic_free,reduced_sugar,vegetarian,pesticide_free,grain_free,no_added_sugar,salt_reduced,nitrite_free,fed_without_ogm,no_added_salt,no_artificial_flavours,porc,vegan,frozen,fat_free,reduced_fats,fresh,alcool,lactose_free,phenylalanine_free,palm_oil_free,ecoscore,produits_du_monde,regional_product,national_brand,first_price_brand,carrefour_brand
13,Product_51620,33CL LIMONADE LA BEAUCERONNE,Department_10,Class_1000,SubClass_10000,PGC,BEAUCERONN,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0
17,Product_6916,"PET 1,5L LIMONETTE MILLES",Department_10,Class_1000,SubClass_10000,PGC,LIMONETTE,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,1,0,0
18,Product_39850,75CL LORINA LIMONADE CITRON VT,Department_10,Class_1000,SubClass_10000,PGC,LORINA,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0
19,Product_31931,75CL BEAUCERONNE CITRON GLACE,Department_10,Class_1000,SubClass_10000,PGC,BEAUCERONN,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,1,1,0,0
21,Product_1433,BELVOIR CITRONNADE 750ML,Department_10,Class_1000,SubClass_10000,PGC,BELVOIR,Boissons,"Colas, Thés glacés, Sirops et Sodas","Limonades, Limes et Tonics",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82960,Product_57956,885558 CUSHION 4 BLANC CAS2 TU,Department_65,Class_6573,SubClass_65730,TEXTILE,TEX HOME,Maison et Décoration,Mobilier et Décoration,Décorations et Parfums d'intérieur,Coussins et galettes de chaise,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82961,Product_18949,857699 BONNET POLAIRE NOIR TU,Department_66,Class_6620,SubClass_66204,TEXTILE,TEX,Mode et Bagagerie,Bagagerie et Maroquinerie,Accessoires de mode,"Chapeaux, Casquettes et Bonnets",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82962,Product_66524,857699 BONNET PO BLANC CAS2 TU,Department_66,Class_6620,SubClass_66204,TEXTILE,TEX,Mode et Bagagerie,Bagagerie et Maroquinerie,Accessoires de mode,"Chapeaux, Casquettes et Bonnets",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1
82963,Product_66126,876946 PULL DE NOEL PET ECRU S,Department_66,Class_6635,SubClass_66358,TEXTILE,TEX,Animalerie,Chiens,"Colliers, Laisses et Vêtements",Vêtements et Chaussures,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0,0,1


In [21]:
# Load product embeddings
product_embeddings_np = np.load(f'{trainFolder}/product_embeddings.npy')

# Load product IDs
product_ids = np.load(f'{trainFolder}/product_ids.npy', allow_pickle=True)  # Use allow_pickle=True if product_ids is an array of objects (e.g., strings)

# Create the product embedding dictionary
product_embedding_dict = dict(zip(product_ids, product_embeddings_np))

# Negative Sampling using Embeddings

In [22]:
train_data

,customer_id,product_id,is_promo,quantity,recency,month,day_of_week,is_weekend,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products
0,Household_10188,Product_62613,0,1.0,351,1,5,1,346,447,1.069351,0.013423,225
1,Household_10188,Product_71119,0,1.0,581,5,6,1,576,447,1.069351,0.013423,225
2,Household_10188,Product_23971,0,2.0,581,5,6,1,576,447,1.069351,0.013423,225
3,Household_10188,Product_3907,0,1.0,624,4,5,1,619,447,1.069351,0.013423,225
4,Household_10188,Product_37906,0,1.0,21,12,6,1,16,447,1.069351,0.013423,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756541,Household_19505,Product_62926,0,1.0,499,8,4,0,490,381,1.094488,0.028871,270
8756542,Household_11112,Product_75788,0,1.0,75,10,1,0,50,370,1.186486,0.029730,227
8756543,Household_11112,Product_46620,0,1.0,36,11,5,1,11,370,1.186486,0.029730,227
8756544,Household_11112,Product_27326,0,1.0,177,7,4,0,152,370,1.186486,0.029730,227


In [23]:
len(train_data)

8756546

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import pandas as pd

# Generate a list of all unique product IDs
all_product_ids = set(product_embedding_dict.keys())

# Prepare for storing negative samples
negative_samples = []

# Group by customer for efficient processing
customer_groups = train_data.groupby('customer_id')

for customer_id, group in tqdm(customer_groups, desc="Generating hard negative samples"):
    # Purchased products for the current customer
    purchased_products = set(group['product_id'])

    # Non-purchased products for the current customer
    non_purchased_products = list(all_product_ids - purchased_products)

    # Skip if there are no products to compare
    if len(non_purchased_products) == 0 or len(purchased_products) == 0:
        continue

    # Get embeddings for purchased and non-purchased products
    purchased_embeddings = np.array([product_embedding_dict[pid] for pid in purchased_products])
    non_purchased_embeddings = np.array([product_embedding_dict[pid] for pid in non_purchased_products])

    # Reshape to ensure 2D arrays
    purchased_embeddings = purchased_embeddings.reshape(-1, purchased_embeddings.shape[-1])
    non_purchased_embeddings = non_purchased_embeddings.reshape(-1, non_purchased_embeddings.shape[-1])

    # Compute cosine similarity between non-purchased and purchased products
    similarity_scores = cosine_similarity(non_purchased_embeddings, purchased_embeddings)

    # Identify the most similar non-purchased products (hard.  negatives)
    most_similar_indices = np.argsort(-similarity_scores, axis=0)[:len(group)]
    #hard_negatives = {non_purchased_products[i] for i in most_similar_indices.flatten()}[:len(group)]
    # Flatten and select up to len(group) unique hard negatives
    hard_negatives = list({non_purchased_products[i] for i in most_similar_indices.flatten()})[:len(group)]


    # Append hard negatives as negative samples
    negative_samples.extend([{
        'customer_id': customer_id,
        'product_id': product_id,
        'quantity': 0  # Indicating negative sample
    } for product_id in hard_negatives])

# Convert negative samples to a DataFrame
negative_samples_df = pd.DataFrame(negative_samples)

# Append negative samples to the original training data
train_data = pd.concat([train_data, negative_samples_df], ignore_index=True)

# Update relevance column (1 for purchased, 0 for negatives)
train_data['relevance'] = (train_data['quantity'] > 0).astype(int)


Generating hard negative samples: 100%|██████████| 9999/9999 [9:50:43<00:00,  3.54s/it]


In [ ]:
train_data.tail()

,customer_id,product_id,is_promo,quantity,recency,month,day_of_week,is_weekend,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,unique_products,relevance
17513087,Household_19999,Product_56904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17513088,Household_19999,Product_29298,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17513089,Household_19999,Product_81672,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17513090,Household_19999,Product_31465,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17513091,Household_19999,Product_72672,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Check for missing values
print("Missing values in train_data:")
print(train_data.isnull().sum())

print("Missing values in test_data:")
print(test_data.isnull().sum())

# Fill remaining missing values with zeros as a safety measure
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

Missing values in train_data:
customer_id                       0
product_id                        0
is_promo                    8756546
quantity                          0
recency                     8756546
month                       8756546
day_of_week                 8756546
is_weekend                  8756546
days_since_last_purchase    8756546
purchase_frequency          8756546
avg_quantity                8756546
promo_ratio                 8756546
unique_products             8756546
relevance                         0
dtype: int64
Missing values in test_data:
transaction_id    0
customer_id       0
product_id        0
dtype: int64


In [ ]:
train_data = train_data.drop(columns=['day_of_week','is_weekend','unique_products'])

In [ ]:
# prompt: in train_data for household Household_9247 count how manyrelevance 1 and how many are 0.

# Assuming 'train_data' DataFrame is already loaded as in the provided code.

household_9247_data = train_data[train_data['customer_id'] == 'Household_76806']

relevance_counts = household_9247_data['relevance'].value_counts()

print(f"Relevance counts for Household_9247:")
relevance_counts

Relevance counts for Household_9247:


,count
relevance,


In [ ]:

train_data.to_csv(os.path.join(trainFolder, 'train_data_all_2.csv'), index=False)

In [ ]:
len(train_data)

17513092

In [ ]:
train_data

,customer_id,product_id,is_promo,quantity,recency,month,days_since_last_purchase,purchase_frequency,avg_quantity,promo_ratio,relevance
0,Household_10188,Product_62613,0.0,1.0,351.0,1.0,346.0,447.0,1.069351,0.013423,1
1,Household_10188,Product_71119,0.0,1.0,581.0,5.0,576.0,447.0,1.069351,0.013423,1
2,Household_10188,Product_23971,0.0,2.0,581.0,5.0,576.0,447.0,1.069351,0.013423,1
3,Household_10188,Product_3907,0.0,1.0,624.0,4.0,619.0,447.0,1.069351,0.013423,1
4,Household_10188,Product_37906,0.0,1.0,21.0,12.0,16.0,447.0,1.069351,0.013423,1
...,...,...,...,...,...,...,...,...,...,...,...
17513087,Household_19999,Product_56904,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
17513088,Household_19999,Product_29298,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
17513089,Household_19999,Product_81672,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0
17513090,Household_19999,Product_31465,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0


In [ ]:
products_data